In [1]:
import json
import gzip
import os
from collections import Counter
import datetime
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sp
import pdb

In [2]:
## read JSON data
# change the file path accordingly
# test data: file from wget http://data.gharchive.org/2018-07-16-15.json.gz
# i.e. archive for 24 hours on 7/16/18

######### CHANGE ME #############
# specify date and hours for the archive data; assumes that the data are downloaded in the given path
# also assumes the data is from july, 2018
my_path = '/Users/amy/git/MetaGithub/data/' 
date_start = 16 # inclusive
date_end = 17 # exclusive
hour_start = 15 # inclusive
hour_end = 16 # exclusive
#################################

events = []

# for every date
for date in range(date_start, date_end):
    
    # format date string so that 1 is encoded as '01', etc.
    my_date = str(date) if date > 9 else '0'+ str(date)
    
    # for every hour
    for hour in range(hour_start, hour_end):
        
        with gzip.open('%s2018-07-%s-%d.json.gz' %(my_path, my_date, hour)) as f:
            for line in f:
                events.append(json.loads(line.strip()))

NameError: name 'gzip' is not defined

In [1]:
## get all watch events
star_events = [event for event in events if event['type'] == 'WatchEvent']

# aggregate starred repos for each user
user2repo = {} # maps user name to list of repo names they've starred
repos = set() # set of all repos that were starred
user2id = {} # maps user name to its index
repo2id = {} # maps repo name to its index
user_id = 0 # user index starts at 0
repo_id = 0 # repo index starts at 0

for event in star_events:
    
    repo_name = event['repo']['name']
    user_name = event['actor']['login']
    
    # for new user, update user2id and user_id
    if user_name not in user2repo:
        user2repo[user_name] = [repo_name]
        user2id[user_name] = user_id
        user_id += 1
    else:
        user2repo.get(user_name).append(repo_name)
        
    # for new repo, update repo2id and repo_id
    if repo_name not in repos:
        repos.add(repo_name)
        repo2id[repo_name] = repo_id
        repo_id += 1
    

NameError: name 'events' is not defined

In [4]:
## make a sparse matrix where entry (i,j) is 1 if the user i starred repo j
n_user = len(user2repo)
n_repo = len(repos)

# initialie spare matrix
mat = sp.dok_matrix((n_user,n_repo), dtype=np.int8)

# set (i,j) = 1 if user i starred repo j
for user_name, repo_names in user2repo.items():
    
    user_id = user2id[user_name]
    repo_ids = [repo2id[r] for r in repo_names]

    mat[user_id, repo_ids] = 1

# convert to csr (compressed row sparse) matrix
mat = mat.tocsr()
print(mat.shape)

(4092, 4391)


In [5]:
# build dataframes we want to save: (i, users), (j, repos)

# not efficient but whatever
users = ['' for i in range(0, len(user2id))]
for u, i in user2id.items():
    users[i] = u

repos = ['' for i in range(0, len(repo2id))]
for r, i in repo2id.items():
    repos[i] = r

df_users = pd.DataFrame({'index': range(0, len(users)), 'user' : users})
df_repos = pd.DataFrame({'index': range(0, len(repos)), 'repo' : repos})
df_repos.head()

,index,repo
0,0,didierfranc/react-waterfall
1,1,Vscarlett/JQuery-stoplight
2,2,Vscarlett/Personal-Express-Login
3,3,rickey-g/fancybear
4,4,Vscarlett/Name-Generator


In [8]:
# save files

### CHANGE THIS PATH ACCORDINGLY ###
out_path = '/Users/amy/git/MetaGithub/'

# save users, repos, matrix
df_users.to_csv(out_path + 'user_list.csv')
df_repos.to_csv(out_path + 'repo_list.csv')
sp.save_npz(out_path+"user_repo_matrix.npz", mat)

In [70]:
# loading the sparse matrix
test = sp.load_npz(out_path+"user_repo_matrix.npz")